In [35]:
%pip install torch torchvision  # Cài đặt PyTorch và các gói liên quan
%pip install scikit-learn  # Cài đặt scikit-learn
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [36]:
import pandas as pd
file_path = 'Data/Doanhthu_chuaCH_2thang.csv'

# Đọc file CSV
df = pd.read_csv(file_path)
# Đếm số hàng trong DataFrame
so_hang = df.shape[0]  # df.shape trả về (số hàng, số cột)
print(f"Số hàng trong bảng: {so_hang}")


# Hiển thị DataFrame
print("Dữ liệu ban đầu:")
print(df.head())

# Hiển thị thông tin tổng quan
print("\nThông tin tổng quan về dữ liệu:")
print(df.info())

# Hiển thị thống kê cơ bản
print("\nThống kê mô tả dữ liệu:")
print(df.describe())

Số hàng trong bảng: 59
Dữ liệu ban đầu:
      Ngày:   Doanh thu Affilicate (nghìn đồng)  \
0  8/10/2024                            1410000   
1  8/11/2024                            1470000   
2  8/12/2024                            1340000   
3  8/13/2024                            1790000   
4  8/14/2024                            1610000   

   Doanh thu cửa hàng (nghìn đồng)  Đơn giá bình quân (nghìn đồng)  \
0                          1940.00                          118.91   
1                          6770.00                          119.14   
2                          3130.00                          104.27   
3                           829.85                          117.69   
4                             0.00                          114.85   

   Lượt bán (nghìn lượt)  Doanh thu tài khoản tự vận hành (nghìn đồng)  
0                  12.35                                         10440  
1                  13.32                                         12500  
2            

In [37]:
%pip install torch

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [38]:
# Loại bỏ khoảng trắng ở đầu và cuối tên cột
df.columns = df.columns.str.strip()
# Bỏ cột không cần thiết


# Chuyển đổi cột 'Ngày:' thành kiểu datetime (nếu cần)
# df['Ngày'] = pd.to_datetime(df['Ngày'])

# Chọn các cột dữ liệu số
df_data = df.iloc[:, 1:]  # Bỏ cột 'Ngày:', chỉ giữ lại các cột dữ liệu số

# Chuyển đổi tất cả các cột thành kiểu số (nếu cần)
df_data = df_data.apply(pd.to_numeric, errors='coerce')

# Xử lý giá trị NaN: Thay thế NaN bằng giá trị trung bình
df_data = df_data.fillna(df_data.mean())

# Kiểm tra lại dữ liệu sau khi chuyển đổi
print(df_data.head())
print("Số lượng giá trị NaN còn lại trong mỗi cột:\n", df_data.isnull().sum())

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(df_data)  # Chuẩn hóa các cột dữ liệu

# Tạo tập dữ liệu với nhiều biến đầu vào
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :])  # Lấy tất cả các cột
        y.append(data[i + time_step, :])  # Dự đoán tất cả các cột
    return np.array(X), np.array(y)

# Số bước thời gian của chuỗi
time_step = 10  # Có thể điều chỉnh nếu cần
X, y = create_dataset(data, time_step)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])  # Giữ nguyên số lượng cột

# Tạo mô hình LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=50, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # Số cột output có thể khác với số cột input

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Lấy đầu ra của bước cuối cùng
        return out

# Khởi tạo mô hình, loss và optimizer
input_size = X.shape[2]  # Số lượng cột đầu vào
output_size = X.shape[2]  # Dự đoán tất cả các cột
model = LSTMModel(input_size=input_size, output_size=output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_size = int(len(X) * 0.7)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Huấn luyện mô hình
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(torch.FloatTensor(X_train))
    loss = criterion(outputs, torch.FloatTensor(y_train))
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Dự đoán cho tập kiểm tra
model.eval()
test_predict = model(torch.FloatTensor(X_test)).detach().numpy()


# Tính toán các chỉ số hồi quy (MSE, RMSE, MAE, R²)
mse = mean_squared_error(y_test, test_predict)  # Mean Squared Error
rmse = np.sqrt(mse)  # Root Mean Squared Error
mae = mean_absolute_error(y_test, test_predict)  # Mean Absolute Error
r2 = r2_score(y_test, test_predict)  # R² score

# In ra các chỉ số hồi quy
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R² Score: {r2:.4f}")
# Dự đoán 3 ngày tiếp theo (dựa trên dữ liệu cuối cùng trong tập test)
input_data = X_test[-1].reshape(1, time_step, input_size)  # Dữ liệu cuối cùng trong tập test
predictions = []
for _ in range(3):  # Dự đoán cho 3 ngày tiếp theo
    pred = model(torch.FloatTensor(input_data)).detach().numpy()
    predictions.append(pred)
    # Cập nhật input_data với dự đoán mới
    input_data = np.append(input_data[:, 1:, :], pred.reshape(1, 1, input_size), axis=1)

# Chuyển predictions thành numpy array và ngược lại thang đo gốc
predictions = np.array(predictions).reshape(3, output_size)  # 3 ngày, số lượng cột bằng với output
predictions_original_scale = scaler.inverse_transform(predictions)

# In dự đoán đã được chuyển đổi về thang đo gốc
print("Dự đoán cho 3 ngày tiếp theo (giá trị gốc): ", predictions_original_scale)

   Doanh thu Affilicate (nghìn đồng)  Doanh thu cửa hàng (nghìn đồng)  \
0                            1410000                          1940.00   
1                            1470000                          6770.00   
2                            1340000                          3130.00   
3                            1790000                           829.85   
4                            1610000                             0.00   

   Đơn giá bình quân (nghìn đồng)  Lượt bán (nghìn lượt)  \
0                          118.91                  12.35   
1                          119.14                  13.32   
2                          104.27                  13.50   
3                          117.69                  15.97   
4                          114.85                  14.81   

   Doanh thu tài khoản tự vận hành (nghìn đồng)  
0                                         10440  
1                                         12500  
2                                         13060  


In [39]:
%pip install torch

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [40]:
file_path = 'Data/Doanhthu_chuaCH_2thang.csv'
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()
df['Ngày:'] = pd.to_datetime(df['Ngày:'])

# Chuyển cột "Ngày:" sang giá trị số (epoch timestamp)
df['Ngày:'] = df['Ngày:'].map(pd.Timestamp.timestamp)

# Tách thuộc tính và mục tiêu
attribute = df[['Ngày:']].values
targets = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(df.mean()).values

In [41]:
# Chuẩn hóa dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
# attribute = scaler.fit_transform(attribute)
targets = scaler.fit_transform(targets)

# Hàm tạo dữ liệu theo bước thời gian
def create_dataset(attribute, target, time_step=1):
    X, y = [], []
    for i in range(len(attribute) - time_step):
        X.append(attribute[i:(i + time_step), 0])  # Chỉ cột "Ngày"
        y.append(target[i + time_step])          # Giá trị của Target
    return np.array(X), np.array(y)

# Tạo dữ liệu cho mỗi cột Target
time_step = 10
datasets = []
for col_idx in range(targets.shape[1]):
    X, y = create_dataset(attribute, targets[:, col_idx], time_step)
    datasets.append((X.reshape(X.shape[0], X.shape[1], 1), y))  # Thêm trục thứ 3 cho LSTM

# Định nghĩa mô hình LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_sizes, activation=nn.Sigmoid):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_sizes[0], num_layers=1, batch_first=True)
        self.hidden_layers = nn.Sequential(
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[1], hidden_sizes[2]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[2], hidden_sizes[3]),
            nn.ReLU()
        )
        self.fc = nn.Linear(hidden_sizes[3], output_size)
        self.activation = activation()

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.hidden_layers(out[:, -1, :])
        out = self.fc(out)
        return self.activation(out)

# Khởi tạo các tham số mô hình
input_size = 1  # Chỉ một thuộc tính "Ngày"
output_size = 1  # Dự đoán từng Target
hidden_sizes = [10, 20, 30, 100]
model = LSTMModel(input_size=input_size, output_size=output_size, hidden_sizes=hidden_sizes)

# Huấn luyện và đánh giá
num_epochs = 100
batch_size = 16
learning_rate = 0.001

for col_idx, (X, y) in enumerate(datasets):
    print(f"\nĐang huấn luyện cho Target cột {df.columns[col_idx + 1]}")

    kf = KFold(n_splits=5)
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"  Fold {fold + 1}/{kf.n_splits}")

        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
        val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        # Khởi tạo optimizer và hàm mất mát
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)
        criterion = nn.MSELoss()

        # Huấn luyện
        for epoch in range(num_epochs):
            model.train()
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets.unsqueeze(1))  # Thêm chiều cho target
                loss.backward()
                optimizer.step()

            # Đánh giá trên tập validation
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    outputs = model(inputs)
                    val_loss += criterion(outputs, targets.unsqueeze(1)).item()
            val_loss /= len(val_loader)

            if (epoch + 1) % 10 == 0:
                print(f"    Epoch [{epoch + 1}/{num_epochs}], Val Loss: {val_loss:.4f}")

    # Dự đoán và đánh giá trên tập kiểm tra
    model.eval()
    test_predict = model(torch.FloatTensor(X_val)).detach().numpy()
    mse = mean_squared_error(y_val, test_predict)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_val, test_predict)
    r2 = r2_score(y_val, test_predict)

    print(f"  Kết quả cột {df.columns[col_idx + 1]}:")
    print(f"    Mean Squared Error (MSE): {mse:.4f}")
    print(f"    Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"    Mean Absolute Error (MAE): {mae:.4f}")
    print(f"    R² Score: {r2:.4f}")



Đang huấn luyện cho Target cột Doanh thu Affilicate (nghìn đồng)
  Fold 1/5
    Epoch [10/100], Val Loss: 0.1406
    Epoch [20/100], Val Loss: 0.1772
    Epoch [30/100], Val Loss: 0.1767
    Epoch [40/100], Val Loss: 0.1863
    Epoch [50/100], Val Loss: 0.1759
    Epoch [60/100], Val Loss: 0.1817
    Epoch [70/100], Val Loss: 0.1731
    Epoch [80/100], Val Loss: 0.1784
    Epoch [90/100], Val Loss: 0.1764
    Epoch [100/100], Val Loss: 0.1761
  Fold 2/5
    Epoch [10/100], Val Loss: 0.1017
    Epoch [20/100], Val Loss: 0.1008
    Epoch [30/100], Val Loss: 0.1023
    Epoch [40/100], Val Loss: 0.0996
    Epoch [50/100], Val Loss: 0.1007
    Epoch [60/100], Val Loss: 0.1001
    Epoch [70/100], Val Loss: 0.1040
    Epoch [80/100], Val Loss: 0.1005
    Epoch [90/100], Val Loss: 0.0992
    Epoch [100/100], Val Loss: 0.1008
  Fold 3/5
    Epoch [10/100], Val Loss: 0.0032
    Epoch [20/100], Val Loss: 0.0041
    Epoch [30/100], Val Loss: 0.0044
    Epoch [40/100], Val Loss: 0.0041
    Epoch [